In [1]:
import requests
import pandas as pd
import utils.misc as misc
from config import *
import yahoo_fin.stock_info as si
from yahooquery import Ticker
import quandl
import os
from datetime import datetime, timedelta
from finvizfinance.screener.custom import Custom
import time
from glob import glob

In [2]:
BASE_URL = r"https://www.alphavantage.co/query?"
# av_api = 'CQLP7J9VZG14Y7O4'
av_api2 = 'IHLNUQ0G66C0MJID' #basil

In [3]:
request_times = [0] * 180

In [28]:
def av_api(func, api_key, symbol):
    assert symbol is not None
    symbol = symbol.strip().upper()

    url = f"{BASE_URL}function={func}&symbol={symbol}&apikey={api_key}"
    while True:
        if time.time() - request_times[-180] <= 60:
            time.sleep(1/3)
        else:
            response = requests.get(url)
            request_times.append(time.time())
            break
    try:
        if func == 'EARNINGS':
            return pd.DataFrame(response.json()['quarterlyEarnings'])
            # except:
            #     print(response.json())
            #     output = None
        elif func == 'BALANCE_SHEET':
            # print(response)
            # print(response.json())
            return pd.DataFrame(response.json()['quarterlyReports'])
        elif func == 'INCOME_STATEMENT':
            print(response.json())
            return pd.DataFrame(response.json()['quarterlyReports'])
    except:
        print(response.json())
        return 

In [29]:
def get_av_data(symbol, api_key):
    #For reported Earnings & Analyst Estimated EPS, totalRevenue, commonStockSharesOutstanding
    df1 = av_api('EARNINGS',api_key, symbol)
    # print(df1)
    df2 = av_api('INCOME_STATEMENT',api_key, symbol)[['fiscalDateEnding','totalRevenue']]
    print(df2)
    df1 = df1.merge(df2, on='fiscalDateEnding', how='left')
    df3 = av_api('BALANCE_SHEET',api_key, symbol)[['fiscalDateEnding','commonStockSharesOutstanding']]
    df1 = df1.merge(df3, on='fiscalDateEnding', how='left')

    return df1

In [6]:
stocks = pd.read_csv('../data/remaining.csv', header=None)

In [7]:
us_stocks = pd.read_csv('../data/us_stocks.csv')

In [8]:
os.chdir(ep)

In [9]:
os.chdir('data')

In [10]:
os.getcwd()

'/Users/yash/Desktop/Trading/EP/data'

In [23]:
done_files = glob('*')

In [24]:
for s in stocks[1]:
    outfile = fr'{s}_DATA.csv'
    industry = us_stocks[us_stocks['Ticker'] == s].iloc[0].Industry
    # print(str(industry))
    if (industry == 'Asset Management'):
        # print('Skipping asst mgmt', s)
        continue
    if outfile in done_files:
        # print('Already present', s)
        continue
    tries = 0
    # while True and tries < 5:
    #     tries += 1
    try:
        # print(s)
        df = get_av_data(s, av_api2)
        df.to_csv(outfile, mode='w', sep=',', header=df.columns)
        break
    except Exception as e:
        print("An exception occured: {}".format(e), s, industry)
        time.sleep(1/3)
        continue

An exception occured: 'fiscalDateEnding' TCBIO Banks - Regional
An exception occured: 'fiscalDateEnding' MBINN Banks - Regional
An exception occured: 'fiscalDateEnding' JZ Education & Training Services
An exception occured: 'fiscalDateEnding' HSAI Auto Parts
An exception occured: "None of [Index(['fiscalDateEnding', 'totalRevenue'], dtype='object')] are in the [columns]" GRRR Software - Infrastructure
An exception occured: "None of [Index(['fiscalDateEnding', 'totalRevenue'], dtype='object')] are in the [columns]" CRGO Integrated Freight & Logistics
An exception occured: 'fiscalDateEnding' HKD Software - Application
An exception occured: 'fiscalDateEnding' IXHL Drug Manufacturers - Specialty & Generic
{'Error Message': 'Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for EARNINGS.'}
{'Error Message': 'Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for INCOME_STATEMENT.'}
An ex

In [30]:
get_av_data('GRRR', av_api2)

{'symbol': 'GRRR', 'annualReports': [{'fiscalDateEnding': '2021-12-31', 'reportedCurrency': 'USD', 'grossProfit': '15774201', 'totalRevenue': '42242863', 'costOfRevenue': '26468662', 'costofGoodsAndServicesSold': '26468662', 'operatingIncome': '-8158259', 'sellingGeneralAndAdministrative': '3430230', 'researchAndDevelopment': '15053175', 'operatingExpenses': '23445044', 'investmentIncomeNet': 'None', 'netInterestIncome': '-628480', 'interestIncome': '37869', 'interestExpense': '666349', 'nonInterestIncome': 'None', 'otherNonOperatingIncome': 'None', 'depreciation': 'None', 'depreciationAndAmortization': '8747008', 'incomeBeforeTax': '-8786739', 'incomeTaxExpense': '-238445', 'interestAndDebtExpense': 'None', 'netIncomeFromContinuingOperations': '-8548294', 'comprehensiveIncomeNetOfTax': 'None', 'ebit': '-8120390', 'ebitda': '626618', 'netIncome': '-8548294'}, {'fiscalDateEnding': '2020-12-31', 'reportedCurrency': 'USD', 'grossProfit': '18555388', 'totalRevenue': '45412589', 'costOfReve

KeyError: "None of [Index(['fiscalDateEnding', 'totalRevenue'], dtype='object')] are in the [columns]"

In [13]:
def get_ex_dates(symbol):
    symbol = 'META'
    tkr = Ticker(symbol)
    df = tkr.sec_filings
    df = df[(df['type'] == '10-Q') | ((df['type'] == '10-K'))]
    df.reset_index(inplace=True)
    df = df[['date','title']]
    df.rename(columns={'date': 'Ex-Date'}, inplace=True)
    return df

In [ ]:
av_api('BALANCE_SHEET', av_api, symbol)

In [ ]:
import yfinance as yf
from yahoofinancials import YahooFinancials

In [ ]:
yahoo_financials = YahooFinancials('AAPL')
data = yahoo_financials.get_historical_price_data(start_date='2019-01-01', 
                                                  end_date='2019-12-31', 
                                                  time_interval='weekly')
aapl_df = pd.DataFrame(data['AAPL']['prices'])
aapl_df = aapl_df.drop('date', axis=1).set_index('formatted_date')
aapl_df.head()

In [ ]:
from __future__ import print_function
import time
import intrinio_sdk as intrinio
from intrinio_sdk.rest import ApiException

intrinio.ApiClient().set_api_key('YOUR_API_KEY')
intrinio.ApiClient().allow_retries(True)

identifier = 'AAPL'
page_size = 100
next_page = ''

response = intrinio.ZacksApi().get_zacks_analyst_ratings(identifier=identifier, page_size=page_size, next_page=next_page)
print(response)
    

##IGNORE FOR NOW

    NASDAQ DATALINK - FOR SHORT INTEREST

In [ ]:
ndl_api = 'dys7Lqv_HziWD1ZFk4ry'

In [ ]:
quandl.ApiConfig.api_key = ndl_api

In [ ]:
os.chdir('/Users/yash/Desktop/Trading/intern/Basil/finra_short_interest')

In [ ]:
df2 = pd.read_csv('equityshortinterest_2023_02_26_05_44_02.csv')

Q4 Earnings 2022

In [ ]:
df1.to_csv('us_stocks.csv')

In [ ]:
earnings_df = pd.DataFrame(columns=['Symbol', 'Ex-Date', 'title'])

In [ ]:
tickers = df1['Ticker']
for t in tickers:
    dft = get_ex_dates(t)
    dft["Date"] = dft["Ex-Date"].apply(
        lambda x: datetime.strptime(x, "%Y-%m-%d")
    )
    dft = dft[dft['Date'] >= datetime(2023,1,1)]
    if dft.shape[0] > 0:
        earnings_df.loc[len(earnings_df.index)] = [t]+list(dft.iloc[0,1:]) 

In [ ]:
import csv
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
CSV_URL = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=IBM&interval=1min&slice=year1month1&apikey={av_api}'

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        print(row)

In [ ]:
import requests
from bs4 import BeautifulSoup

headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:105.0) Gecko/20100101 Firefox/105.0"
}
url = f"https://finance.yahoo.com/quote/MSFT/analysis?p=MSFT"
soup = BeautifulSoup(requests.get(url, headers=headers).content, "html5lib")

for table in soup.select("table"):
    th_row = [th.text for th in table.find_all("th")]
    print(th_row)

    for tr in table.select("tr:has(td)"):
        td_row = [td.text for td in tr.find_all("td")]
        print(td_row)

    print()

In [ ]:
df2,e = ek.get_data('GOOGL.O',['TR.RevenueActValue.date','TR.RevenueActValue','TR.EPSActValue'],parameters = {'SDate':'0','EDate':'-5','Period':'FQ0','Frq':'FQ'}) 